In [1]:
from storage_utils.patients_data_new import patients

from learning_pipelines import get_learning_results
import sklearn.svm as svm
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.naive_bayes import GaussianNB
import pandas as pd


from classification.keras_neural_nets import *

import os

/home/franciscosargo/.virtualenvs/pre_epi_seizures/local/lib/python2.7/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# Labeling structure
label_struct = {
                'inter_ictal':{
                    'label': 'Inter-Ictal Data Points',
                    'color': 'blue',
                    'intervals_samples': [(1000 * 0 * 60, 1000 * 20 * 60 )]
                             },
                'pre_ictal':{
                    'label': 'Pre-Ictal data points',
                    'color': 'yellow',
                    'intervals_samples': [(1000 * 20 * 60, 1000 * 49 * 60 )]
                             },
                'ictal':{
                    'label': 'Ictal data points',
                    'color': 'red',
                    'intervals_samples': [(1000 * 50 * 60, 1000 * 55 * 60 )]
                         },
                        }
baseline_label_struct = {
                 'baseline':{
                    'label': 'Baseline Data Points',
                    'color': 'green',
                    'intervals_samples': [(0, 1000 * 30 * 60 )]
                            },
                        }



param_grid = [
              {'ANN__epochs': [2000],
               'ANN__batch_size': [100],
               'ANN__hidden_layers_nr': [i for i in xrange(1, 5)],
               'ANN__hidden_nodes_per_layer': [2**i for i in xrange(1, 5)],
               'ANN__hidden_nodes_activation': ['relu']
              }
]

pipe = Pipeline( [('ANN', KerasClassifier(build_fn=dense_network,
                                        input_dim = 5,
                                          output_dim = 3,
                                          verbose=0))])

param_grid = [
               {'GaussNB__priors': [None]},
                  ]
pipe = Pipeline([('GaussNB', GaussianNB())])


param_grid = [
               {'SVC__C': [2**i for i in xrange(-5, 11)],
                'SVC__gamma':[2**i for i in xrange(-15, 1)]},
                 ]

pipe = Pipeline([('SVC', svm.SVC())])



feature_slot = 'pca_beat_amp_computation'
hyper_param=0


scaler = StandardScaler()

plot_eda=False
learn_flag=True
compute_all_new=True

In [3]:
# Perform Hyper analysis

patient_lists = [[3], [4]]


table = [get_learning_results(label_struct, baseline_label_struct,
                                              pipe, scaler, param_grid,
                                              patient_list,
                                              feature_slot,
                                              hyper_param,
                                              plot_eda,
                                              learn_flag,
                                              compute_all_new)
                   for patient_list in patient_lists]
    

('scaler', StandardScaler(copy=True, with_mean=True, with_std=True))('lead_list', ['ECG-'])('feature_slot', 'pca_beat_amp_computation')('interim_processing', [StandardScaler(copy=True, with_mean=True, with_std=True)])('assign_baseline', 'assign_equal_baseline_seizure')('label_struct', {'pre_ictal': {'color': 'yellow', 'intervals_samples': [(1200000, 2940000)], 'label': 'Pre-Ictal data points'}, 'inter_ictal': {'color': 'blue', 'intervals_samples': [(0, 1200000)], 'label': 'Inter-Ictal Data Points'}, 'ictal': {'color': 'red', 'intervals_samples': [(3000000, 3300000)], 'label': 'Ictal data points'}})('baseline_label_struct', {'baseline': {'color': 'green', 'intervals_samples': [(0, 1800000)], 'label': 'Baseline Data Points'}})('patient_list', [3])('hyper_param', 0)('group_id', 'seizure_nr')

This is a list of the parameters
["('scaler', StandardScaler(copy=True, with_mean=True, with_std=True))('lead_list', ['ECG-'])('feature_slot', 'pca_beat_amp_computation')('interim_processing', [Stand

   params     id
0    True  False

final id -- check disk
5d9f23b0-4640-11e8-894a-b06ebfc25cc2

Check for this path on disk .. should be there
5d9f23b0-4640-11e8-894a-b06ebfc25cc2
('search_function', <class 'sklearn.model_selection._search.GridSearchCV'>)('scoring', ['f1_micro'])('cv_in', LeavePGroupsOut(n_groups=1))('label', 'label')('pipe', "['SVC']")('cv_out', LeavePGroupsOut(n_groups=1))('param_grid', [{'SVC__gamma': [3.0517578125e-05, 6.103515625e-05, 0.0001220703125, 0.000244140625, 0.00048828125, 0.0009765625, 0.001953125, 0.00390625, 0.0078125, 0.015625, 0.03125, 0.0625, 0.125, 0.25, 0.5, 1], 'SVC__C': [0.03125, 0.0625, 0.125, 0.25, 0.5, 1, 2, 4, 8, 16, 32, 64, 128, 256, 512, 1024]}])

This is a list of the parameters
['(\'search_function\', <class \'sklearn.model_selection._search.GridSearchCV\'>)(\'scoring\', [\'f1_micro\'])(\'cv_in\', LeavePGroupsOut(n_groups=1))(\'label\', \'label\')(\'pipe\', "[\'KNN\']")(\'cv_out\', LeavePGroupsOut(n_groups=1))(\'param_grid\', [{\'KNN__n_

IOError: File /mnt/Seagate/pre_epi_seizures/EDanalysis_new/812fee52-4647-11e8-894a-b06ebfc25cc2/346c76aa-4659-11e8-894a-b06ebfc25cc2/classification_report.h5 does not exist

In [ ]:
table = pd.concat(table)
table

In [ ]:
def apply_post_learning_labels(table, patients):
    
    table
    print patients

In [ ]:
apply_post_learning_labels(table, patients)

In [ ]:

per_patient = table[['precision',
                     'recall',
                     'f1-score',
                     'patient_nr',
                     'types_of_seizure',
                     'Labels']].groupby(['types_of_seizure', 'Labels']).describe()

#print per_patient.columns.get_level_values(1)=='std'

#per_patient = per_patient.iloc[:,
                               #per_patient.columns.get_level_values(1)=='std']
per_patient

In [ ]:
# Organize results by label

per_patient.loc[:, (['precision', 'recall', 'f1-score'], ['mean', 'std'])]

In [ ]:
# 

per_patient = table[['precision',
                     'recall',
                     'f1-score',
                     'patient_nr',
                     'Labels',
                     'types_of_seizure',
                     'location']].groupby(['patient_nr',
                                           'Labels',
                                           'types_of_seizure',
                                           'location']).describe()

per_patient_per_seizure_df = per_patient.loc[:, (['precision', 'recall', 'f1-score'], ['mean', 'std'])]

per_patient_per_seizure_df



In [ ]:
# Convert to latex
per_patient_per_seizure_latex = per_patient_per_seizure.to_latex()

per_patient_per_seizure_latex


In [ ]:
print per_patient_per_seizure_latex

In [ ]:
disk = '/mnt/Seagate/pre_epi_seizures/'

latex_dir = 'latex_results/'

results_text = 'results.txt'

full_latex_path = disk + latex_dir + results_text

full_latex_path

In [ ]:
opening = 

In [ ]:
with open(full_latex_path, "w") as file_open:
    
    # Firstly write the feature slot
    file_open.write(feature_slot + '\n')
    file_open.write(per_patient_per_seizure_latex)
